In [1]:
#
# Store Kafka JSON data to a parquet file on HDFS
#

from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType, TimestampType, LongType
from pyspark.sql import functions
from hops import kafka
from hops import tls
from hops import hdfs

# Setup the Schema for storing the data
schema = StructType([
    StructField("observation", StructType([
        StructField("value", StringType(), True),
        StructField("quantityKind", StringType(), True),
        StructField("sensorId", StringType(), True),
        StructField("observationTime", StringType(), True),
    ]), True),
    StructField("sensor", StringType(), True)])

# Setup Topic namnet
TOPIC_NAME = "kringlan-topic-1"
config = kafka.get_kafka_default_config()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
94,application_1638826983412_0036,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [4]:
# Setup Spark Kafka stream
dfk = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka.get_broker_endpoints()) \
  .option("kafka.security.protocol",kafka.get_security_protocol()) \
  .option("kafka.ssl.truststore.location", tls.get_trust_store()) \
  .option("kafka.ssl.truststore.password", tls.get_key_store_pwd()) \
  .option("kafka.ssl.keystore.location", tls.get_key_store()) \
  .option("kafka.ssl.keystore.password", tls.get_key_store_pwd()) \
  .option("kafka.ssl.key.password", tls.get_trust_store_pwd()) \
  .option("kafka.ssl.endpoint.identification.algorithm", "") \
  .option("subscribe", TOPIC_NAME) \
  .load()

# If in need of reading from start!
#  .option("startingOffsets", "earliest") \

# Stream to Parquet fil
PARQ_PATH  = "/Projects/" + hdfs.project_name() + "/Jupyter/Data/eventhub-stream.parquet"
CHECK_PATH = "/Projects/" + hdfs.project_name() + "/Jupyter/Data/eventhub-stream-checkpoint/"

df_output = dfk \
        .selectExpr("CAST(value AS STRING)") \
        .select(functions.from_json("value", schema=schema).alias("data"))

# parquet sink example - will store just once!
targetParquetHDFS = df_output \
    .writeStream \
    .format("parquet") \
    .outputMode("append")\
    .option("path", PARQ_PATH) \
    .option("checkpointLocation", CHECK_PATH) \
    .trigger(once=True) \
    .start()

#    .trigger(processingTime="120 seconds") \

targetParquetHDFS.awaitTermination()